# Exporting satellite images <img align="right" src="../Supplementary_data/dea_logo.jpg">

* [**Sign up to the DEA Sandbox**](https://docs.dea.ga.gov.au/setup/sandbox.html) to run this notebook interactively from a browser
* **Compatibility:** Notebook currently compatible with both the `DEA Sandbox` and `NCI` environments
* **Products used:** 
[ga_ls5t_ard_3](https://explorer.sandbox.dea.ga.gov.au/ga_ls5t_ard_3),
[ga_ls7e_ard_3](https://explorer.sandbox.dea.ga.gov.au/ga_ls7e_ard_3),
[ga_ls8c_ard_3](https://explorer.sandbox.dea.ga.gov.au/ga_ls8c_ard_3)


## Background



### Digital Earth Australia use case




## Description


***

## Getting started

To run this analysis, run all the cells in the notebook, starting with the "Load packages" cell. 

### Load packages
Import Python packages used for the analysis.

In [1]:
# %matplotlib inline

# from datacube.utils.cog import write_cog

%load_ext autoreload
%autoreload 2


import sys
sys.path.insert(1, '../Supplementary_data/')
from notebookapp_imageexport import run_imageexport_app


/env/lib/python3.8/site-packages/datacube/storage/masking.py:7: DeprecationWarning: datacube.storage.masking has moved to datacube.utils.masking
  warnings.warn("datacube.storage.masking has moved to datacube.utils.masking",


### Analysis parameters

The following cell sets important required parameters for the analysis:

* `output_name`: A name that will be used to name the output filmstrip plot file
* `time_range`: The date range to analyse (e.g. `time_range = ('1988-01-01', '2017-12-31')`)
* `time_step`: This parameter allows us to choose the length of the time periods we want to compare (e.g. `time_step = {'years': 5}` will generate one filmstrip plot for every five years of data in the dataset; `time_step = {'months': 18}` will generate one plot for each 18 month period etc.
Time periods are counted from the first value given in `time_range`.

Optional parameters:

* `tide_range`: This parameter allows you to generate filmstrip plots based on specific ocean tide conditions.
This can be valuable for analysing change consistently along the coast. 
For example, `tide_range = (0.0, 0.2)` will select only satellite images acquired at the lowest 20% of tides; `tide_range = (0.8, 1.0)` will select images from the highest 20% of tides. 
The default is `tide_range = (0.0, 1.0)` which will select all images regardless of tide.
* `resolution`: The spatial resolution to load data. 
The default is `resolution = (-30, 30)`, which will load data at 30 m pixel resolution. 
Increasing this (e.g. to `resolution = (-100, 100)`) can be useful for loading large spatial extents.
* `max_cloud`: This parameter allows you to exclude satellite images with excessive cloud. 
The default is `50`, which will keep all images with less than 50% cloud.
* `ls7_slc_off`: Whether to include data from after the Landsat 7 SLC failure (i.e. SLC-off). 
Defaults to `False`, which removes all Landsat 7 observations after May 31 2003.
Setting this to `True` will result in extra data, but can also introduce horizontal striping in the output filmstrip plots.



**If running the notebook for the first time**, keep the default settings below. This will demonstrate how the analysis works and provide meaningful results. 


In [72]:
# Required parameters
output_name = 'example'
satellites = 'sentinel2_nrt'
date = '2021-09-01'
style = 'false colour'

# Optional parameters
resolution = (-30, 30)
vmin, vmax = (0, 2000)
percentile_stretch = (0.1, 99.9)

## Select location and generate filmstrips
Run the following cell to start the analysis. 
This will plot an interactive map that is used to select the area to load satellite data for.

Select the `Draw a rectangle` or `Draw a polygon` tool on the left of the map, and draw a shape around the area you are interested in.

For the first run, try **drawing a square around Sydney Airport and Port Botany** to see an example of change driven by urban and coastal development.

When you are ready, press the green `done` button on the top right of the map. 
This will start loading the data, and then generate a filmstrips plot.

> Depending on the size of the area you select, this step can take **several minutes to complete**.
To keep load times reasonable, select an area **smaller than 200 square kilometers** in size (this limit can be overuled by supplying the `size_limit` parameter in the `run_filmstrip_app` function below).

> Once the analysis reaches the `Generating geomedian composites` step, you can check the status of the data load by clicking the **Dashboard** link under **Client** below.


In [ ]:
ds = run_imageexport_app(output_name, satellites, date, style, vmin, vmax,
                         percentile_stretch)

Map(center=[-13.345590499999998, 130.058899], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom…

Starting analysis...


/env/lib/python3.8/site-packages/distributed/node.py:151: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 39831 instead
  warnings.warn(


Client Scheduler: tcp://127.0.0.1:45233 Dashboard: /user/robbibt/proxy/39831/status,Cluster Workers: 1 Cores: 7 Memory: 63.57 GB


Finding datasets
    s2a_nrt_granule
    s2b_nrt_granule
Returning 7 time steps as a dask array
Exporting image to file...


In [33]:
from ipyleaflet import basemaps

In [ ]:

# # Keep nearest timestep only
# nearest_time = np.argmin(abs(ds.time.values - np.datetime64(date)))
# nearest_ds = ds.isel(time=nearest_time)

# # Convert to numpy array
# rgb_array = np.transpose(nearest_ds.to_array().values, 
#                          axes=[1, 2, 0])

In [ ]:
# nearest_ds.time

In [37]:
import numpy as np


2

## Using filmstrip plots to identify change

The filmstrip plot above contains several colour imagery panels that summarise the median or 'typical' appearance of the landscape for the time periods defined using `time_range` and `time_step`. 
If you ran the analysis over the Sydney Airport and Port Botany area, inspect each of the imagery plots. 
Some key examples of change that appear include: 

* The 1994 construction of Sydney Airport's third runway in the second panel (covering the five year period between 1993 and 1998)
* The 2011 expansion of Port Botany visible as construction in the fifth panel, and completed by the sixth panel
* Increasing commercial and industrial development (e.g. large white roofs) in the suburb of Mascot north of Port Botany

### Change heatmap

To make it easier to identify areas that have changed between each filmstrip panel, the final panel provides a "change heatmap".
This highlights pixels whose values vary greatly between the panels in the filmstrip plot. 
Bright colours indicate pixels that have changed; dark colours indicate pixels that have remained relatively similar across time.

Compare the "change heatmap" panel against the colour imagery panels. 
You should be able to clearly see Sydney Airport's third runway and the Port Botany port expansion highlighted in bright colours.

> **Technical info:** The "change heatmap" is calculated by first taking a log transform of the imagery data to emphasize dark pixels, then calculating standard deviation across all of the filmstrip panels to reveal pixels that changed over time.

## Downloading filmstrip plot
The high resolution version of the filmstrip plot generated above will be saved to the same location you are running this notebook from (e.g. typically `Real_world_examples`). 
In JupyterLab, use the file browser to locate the image file with a name in the following format: 

`filmstrip_{output_name}_{date_string}_{time_step}.png`

If you are using the **DEA Sandbox**, you can download the image to your PC by right clicking on the image file and selecting `Download`.

## Export GeoTIFF data
It can be useful to export each of the filmstrip panels generated above as GeoTIFF raster files so that they can be loaded into a Geographic Information System (GIS) software for further analysis.
Because the filmstrip panels were generated using the "geomedian" statistic that preserves relationships between spectral bands, the resulting data can be validly analysed in the same way as we would analyse an individual satellite image.

To export the GeoTIFFs, run the following cell then right click on the files in the JupyterLab file browser and select `Download`.

In [ ]:
# Export filmstrip panels
for i, ds in geomedians.groupby('timestep'):
    print(f'Exporting {i} data')
    write_cog(geo_im=ds.to_array(), 
              fname=f'geotiff_{output_name}_{i}.tif', 
              overwrite=True)

# Export change heatmap
print('Exporting change heatmap')
write_cog(geo_im=heatmap, 
          fname=f'geotiff_{output_name}_heatmap.tif', 
          overwrite=True)

## Next steps
When you are done, return to the [Analysis parameters](#Analysis-parameters) section, modify some values and rerun the analysis. 
For example, you could try:

* Modify `time_range` to look at a specific time period of interest (e.g. `time_range = ('1990-01-01', '2000-01-01')`.
* Setting a shorter `time_step` (e.g. `time_step = {'years': 2}`) for a more detailed look at how the landscape has changed over shorter time periods.
* Inspecting change along the coastline after controlling for tide using the `tide_range` parameter (e.g. `tide_range = (0.0, 0.3)` to look at the landscape during the lowest 30% of tides). 
For the best results, test this out in an area with high tides such as [Roebuck Bay in West Australia's Kimberley region](https://goo.gl/maps/ZkCcXUSbsVhRMsgM9).


***

## Additional information

**License:** The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0). 
Digital Earth Australia data is licensed under the [Creative Commons by Attribution 4.0](https://creativecommons.org/licenses/by/4.0/) license.

**Contact:** If you need assistance, please post a question on the [Open Data Cube Slack channel](http://slack.opendatacube.org/) or on the [GIS Stack Exchange](https://gis.stackexchange.com/questions/ask?tags=open-data-cube) using the `open-data-cube` tag (you can view previously asked questions [here](https://gis.stackexchange.com/questions/tagged/open-data-cube)).
If you would like to report an issue with this notebook, you can file one on [Github](https://github.com/GeoscienceAustralia/dea-notebooks).

**Last modified:** June 2020

**Compatible datacube version:** 

In [ ]:
import datacube
print(datacube.__version__)

## Tags
Browse all available tags on the DEA User Guide's [Tags Index](https://docs.dea.ga.gov.au/genindex.html)